In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

intraday = pd.read_csv("../00_data/[backup] intraday.csv", sep=";", index_col=["date", "hours"], decimal=",")
day_ahead = pd.read_csv("../00_data/[backup] day_ahead_auction_longformat.csv", sep=";", decimal=",")
forecasts = pd.read_csv("../00_data/time_series_60min_singleindex_filtered.csv", sep=",", decimal=".")

#print(intraday.head(5))
#print(day_ahead.head(5))
#print(forecasts.head(2))

forecasts["time"] = forecasts["cet_cest_timestamp"].str[:-5] #cut +... part
forecasts["timestamp"] = forecasts["time"].map(lambda x: datetime.strptime(x,'%Y-%m-%dT%H:%M:%S'))

intraday = intraday.iloc[:, 3:4] # use only weight. avg column for intraday
merged_data = intraday.merge(day_ahead, left_on=["date", "hours"], right_on=["date", "hour"])
merged_data = merged_data[["date", "hour", "dayAheadPrice", "Weight. Avg.(€/MWh)"]]
merged_data.columns = ["date", "hour", "day_ahead_price", "intraday_price"]
merged_data["price_diff"] = merged_data["day_ahead_price"]-merged_data["intraday_price"]
merged_data["hour"] = merged_data["hour"].str[1:3]
merged_data["time"] = merged_data[['date', 'hour']].apply(lambda x: ' '.join(x), axis=1)
merged_data["datetime"] = merged_data["time"].map(lambda x: datetime.strptime(x, '%Y-%m-%d %H'))

merged_data = merged_data.merge(forecasts, left_on=["datetime"], right_on=["timestamp"])
merged_data = merged_data[['date','hour', 'datetime', 'day_ahead_price','intraday_price','price_diff','DE_solar_capacity','DE_solar_generation_actual','DE_wind_capacity','DE_wind_generation_actual','DE_wind_offshore_capacity','DE_wind_offshore_generation_actual','DE_wind_onshore_capacity','DE_wind_onshore_generation_actual', 'DE_tennet_wind_generation_forecast', 'DE_tennet_solar_generation_forecast', 'DE_amprion_wind_generation_forecast', 'DE_amprion_solar_generation_forecast', 'DE_50hertz_wind_generation_forecast', 'DE_50hertz_solar_generation_forecast', 'DE_transnetbw_solar_generation_forecast', 'DE_transnetbw_wind_generation_forecast']]
merged_data["solar_generation_forecast"] = merged_data["DE_transnetbw_solar_generation_forecast"]+merged_data["DE_amprion_solar_generation_forecast"]+merged_data["DE_50hertz_solar_generation_forecast"]+merged_data["DE_tennet_solar_generation_forecast"]
merged_data["wind_generation_forecast"] = merged_data["DE_transnetbw_wind_generation_forecast"]+merged_data["DE_amprion_wind_generation_forecast"]+merged_data["DE_50hertz_wind_generation_forecast"]+merged_data["DE_tennet_wind_generation_forecast"]
merged_data = merged_data[['date','hour', 'datetime', 'day_ahead_price','intraday_price','price_diff','DE_solar_capacity','DE_solar_generation_actual','DE_wind_capacity','DE_wind_generation_actual', 'solar_generation_forecast', 'wind_generation_forecast']]
merged_data.columns = ['date','hour', 'datetime', 'day_ahead_price','intraday_price','price_diff','solar_capacity','solar_generation_actual','wind_capacity','wind_generation_actual', 'solar_generation_forecast', 'wind_generation_forecast']
merged_data["total_generation_forecast"] = merged_data["wind_generation_forecast"] + merged_data["solar_generation_forecast"]
merged_data["total_generation_actual"] = merged_data["solar_generation_actual"]+merged_data["wind_generation_actual"]
merged_data["forecast_error"] = merged_data["total_generation_forecast"] - merged_data["total_generation_actual"]
merged_data["forecast_error_percent"] = merged_data["forecast_error"]/merged_data["total_generation_forecast"]
merged_data["pricediff_percent"] = merged_data["price_diff"]/merged_data["day_ahead_price"]
#print(merged_data[merged_data.isnull().any(axis=1)])
merged_data.to_csv("../00_data/merged_all_with_datetime_by_algo.csv", sep=";", decimal=",", index=False)
print("done")

done
